# Automate ML Workflows with Pipelines

There are standard workflows in applied ML. Standard because they overcome common problems like data leakage in your test harness. 

Python scikit-learn provides a Pipeline utility to help automate ML workflows. Pipelines work by allowing for a linear sequence of data transforms to be chained together culminating in a modeling process that can be evaluated.

The goal is to ensure that all of the steps in the pipeline are constrained to the data available for the evaluation, such as the training dataset or each fold of the cross-validation procedure. 

You can learn more about Pipelines in scikit-learn by reading [the Pipeline section of the user guide](http://scikit-learn.org/stable/modules/pipeline.html). You can also review the [API documentation for the Pipeline and FeatureUnion classes and the `pipeline` module](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.pipeline).

## Data Preparation and Modeling Pipeline

An easy trap to fall into in applied ML is leaking data from your training dataset to your test dataset. To avoid this trap you need a robust test harness with strong separation of training and testing. This includes data preparation. 

Data preparation is one easy way to leak knowledge of the whole training dataset to the algorithm. For example, preparing your data using normalization or standardization on the entire training dataset before learning would not be a valid test because the training dataset would have been influenced by the scale of the data in the test set.

Pipelines help you prevent data leakage in your test harness by ensuring that data preparation like standardization is constrained to each fold of your cross-validation procedure. 

The example below demonstrates this important data preparation and model evaluation workflow on the diabetes dataset. The pipeline is defined with two steps:

1. Standardize the data
2. Learn a Linear Discriminant Analysis model.

The pipeline is then evaluated using 10-fold cross-validation.

In [ ]:
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [3]:
# Create a pipeline that standardizes the data then creates a model

# load dataset
filename = 'pima-indians-diabetes.data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]

# create pipeline
estimators = []
estimators.append(( 'standardize' , StandardScaler()))
estimators.append(( 'lda' , LinearDiscriminantAnalysis()))
model = Pipeline(estimators)

# evaluate pipeline
kfold = KFold(n_splits=10, random_state=7)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

0.773462064252


Notice how we create a Python list of steps that are provided to the Pipeline for processing the data.

Also notice how the Pipeline itself is treated like an estimator and is evaluated in its entirety by the k-fold cross-validation procedure. 

Running the example provides a summary of accuracy of the setup on the dataset.

How would it be with no pipelines?

In [2]:
#  NO PIPELINE

# load dataset
filename = 'pima-indians-diabetes.data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]

# scaler..
scaler = StandardScaler().fit(X)
rescaledX = scaler.transform(X)

# LDA Classification
kfold = KFold(n_splits=10, random_state=7)
model = LinearDiscriminantAnalysis()                           # <---
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

0.773462064252


## Feature Extraction and Modeling Pipeline

Feature extraction is another procedure that is susceptible to data leakage. 

Like data preparation, feature extraction procedures must be restricted to the data in your training dataset. The pipeline provides a handy tool called the FeatureUnion which allows the results of multiple feature selection and extraction procedures to be combined into a larger dataset on which a model can be trained. Importantly, all the feature extraction and the feature union occurs within each fold of the cross-validation procedure. The example below demonstrates the pipeline defined with four steps:

1. Feature Extraction with Principal Component Analysis (3 features)
2. Feature Extraction with Statistical Selection (6 features)
3. Feature Union
4. Learn a Logistic Regression Model.

The pipeline is then evaluated using 10-fold cross-validation.

In [ ]:
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest

In [ ]:
# Create a pipeline that extracts features from the data then creates a model

# load dataset
filename = 'pima-indians-diabetes.data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]

# create feature union
features = []
features.append(( 'pca' , PCA(n_components=3)))
features.append(( 'select_best' , SelectKBest(k=6)))
feature_union = FeatureUnion(features)

# create pipeline
estimators = []
estimators.append(( 'feature_union' , feature_union))
estimators.append(( 'logistic' , LogisticRegression()))
model = Pipeline(estimators)

# evaluate pipeline
kfold = KFold(n_splits=10, random_state=7)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

Notice how the FeatureUnion is its own Pipeline that in turn is a single step in the final Pipeline used to feed Logistic Regression. 

.. and yes: this might get you thinking about how you can start embedding pipelines within pipelines..

Running the example provides a summary of accuracy of the setup on the dataset.

## Summary

What we did:

* we discovered the difficulties of data leakage in applied ML. You discovered the Pipeline utilities in Python scikit-learn and how they can be used to automate standard applied ML workflows. You learned how to use Pipelines in two important use cases (Data preparation and modeling constrained to each fold of the cross-validation procedure, and 
Feature extraction and feature union constrained to each fold of the cross-validation procedure).

## What's next 

You will take your first look at how to improve algorithm performance on your problems by using ensemble methods.